In [1]:
import os
import numpy as np
import re




In [2]:
def get_file_array(step_file_path):
    with open(step_file_path, "r") as file:
        fileArray = np.array(file.readlines())

    modifiedArray = np.array([], dtype=int)
    for line in fileArray:
        if "#" in line:
            modifiedArray = np.append(modifiedArray, line)

    fileArray = modifiedArray.copy()
    return fileArray


In [3]:
def get_line_by_hash(step_array,hash_num):
    hash_num = "#" + str(hash_num)
    for line in step_array:
        if line.startswith(hash_num):
            return line
        

In [4]:
def get_element_name(line):
    match = re.search(r'=(\s*)(\w+)', line)
    if match:
        word_after_equal = match.group(2)
        return word_after_equal
    else:
        return False

In [5]:
def get_circle_radius(line):
    match = re.search(r'\(([^,]+),\s*([^,]+),\s*([^,]+)\)', line)
    if match:
        third_element = match.group(3)
        return round(float(third_element),2)
    else:
        return False

In [6]:
def get_circle_center_point(circle_line):
    circle_data= get_extracted_data(step_array,[circle_line])
    center_point_string= get_specific_element(circle_data,"CARTESIAN_POINT")
    points = extract_numbers_from_string(center_point_string[0])
    return points

In [7]:
def get_specific_element(fileArray,element):
    elementArray = np.array([], dtype=int)

    for line in fileArray:
        if element in line:
            elementArray = np.append(elementArray, line.strip())
    return elementArray

In [8]:
def get_inside_hashes(line):
    pattern = r"#(\d+)"
    matches = re.findall(pattern, line)
    numbers = [int(match) for match in matches]
    numbers = numbers[1:]
    return numbers

In [9]:

def data_extract(fileArray,line,output_data):
    hashes = get_inside_hashes(line)
    if hashes:
        for hash1 in hashes:
            new_line = fileArray[hash1 - 1]
            output_data.append(new_line)
            data_extract(fileArray,new_line,output_data)
    return output_data

In [10]:
def get_extracted_data(fileArray,required_array):
    output_data = []
    for line in required_array:
        output_data.append(line)
        output_data=data_extract(fileArray,line,output_data)

    return output_data 

In [11]:
def get_numbers_from_string(string):
    start = string.find("(")
    end = string.find(")")
    parenthesis = string[start:end+1] 
    return parenthesis

In [12]:
def extract_numbers_from_string(string):
    new_string = get_numbers_from_string(string)[1:]
    new_string =  get_numbers_from_string(new_string)
    numbers = new_string[1:-1].split(",")
    numbers = [float(num) for num in numbers]
    return [round(num) for num in numbers]


In [13]:
string = "#339 = CARTESIAN_POINT ( 'NONE',  ( -8.412495441825308330E-15, 0.000000000000000000, -9.999999999999996447 ) ) ;\n"
extract_numbers_from_string(string)

[0, 0, -10]

In [14]:
def identify_major_face(step_array):
    advanced_face_array = get_specific_element(step_array,"ADVANCED_FACE")
    major_face = None
    max_hashes = 0
    for face in advanced_face_array:
        numbers = get_inside_hashes(face)
        if len(numbers)>max_hashes and "T." in face:
            max_hashes = len(numbers)
            major_face = face 
        
    return major_face


In [31]:
step_array = get_file_array("part.STEP")
major_face=identify_major_face(step_array)

In [32]:
output_array= get_extracted_data(step_array,[major_face])
output_array

["#216 = ADVANCED_FACE ( 'NONE', ( #149, #151, #277 ), #59, .T. ) ;",
 "#149 = FACE_OUTER_BOUND ( 'NONE', #310, .T. ) ;\n",
 "#310 = EDGE_LOOP ( 'NONE', ( #22, #197, #373, #303 ) ) ;\n",
 "#22 = ORIENTED_EDGE ( 'NONE', *, *, #271, .T. ) ;\n",
 "#271 = EDGE_CURVE ( 'NONE', #407, #411, #401, .T. ) ;\n",
 "#407 = VERTEX_POINT ( 'NONE', #105 ) ;\n",
 "#105 = CARTESIAN_POINT ( 'NONE',  ( -50.00000000000000000, 6.000000000000000000, -50.00000000000000000 ) ) ;\n",
 "#411 = VERTEX_POINT ( 'NONE', #394 ) ;\n",
 "#394 = CARTESIAN_POINT ( 'NONE',  ( -49.99999999999998579, 6.000000000000000000, 50.00000000000000000 ) ) ;\n",
 "#401 = LINE ( 'NONE', #410, #323 ) ;\n",
 "#410 = CARTESIAN_POINT ( 'NONE',  ( -50.00000000000000000, 6.000000000000000000, -50.00000000000000000 ) ) ;\n",
 "#323 = VECTOR ( 'NONE', #81, 1000.000000000000000 ) ;\n",
 "#81 = DIRECTION ( 'NONE',  ( 1.387778780781445676E-16, 0.000000000000000000, 1.000000000000000000 ) ) ;\n",
 "#197 = ORIENTED_EDGE ( 'NONE', *, *, #133, .T. )

In [34]:
outer_boundary =  get_specific_element(output_array,"FACE_OUTER_BOUND")
outer_boundary

array(["#149 = FACE_OUTER_BOUND ( 'NONE', #310, .T. ) ;"], dtype='<U47')

In [35]:
outer_boundary_data=get_extracted_data(step_array,outer_boundary)

outer_boundary_data

["#149 = FACE_OUTER_BOUND ( 'NONE', #310, .T. ) ;",
 "#310 = EDGE_LOOP ( 'NONE', ( #22, #197, #373, #303 ) ) ;\n",
 "#22 = ORIENTED_EDGE ( 'NONE', *, *, #271, .T. ) ;\n",
 "#271 = EDGE_CURVE ( 'NONE', #407, #411, #401, .T. ) ;\n",
 "#407 = VERTEX_POINT ( 'NONE', #105 ) ;\n",
 "#105 = CARTESIAN_POINT ( 'NONE',  ( -50.00000000000000000, 6.000000000000000000, -50.00000000000000000 ) ) ;\n",
 "#411 = VERTEX_POINT ( 'NONE', #394 ) ;\n",
 "#394 = CARTESIAN_POINT ( 'NONE',  ( -49.99999999999998579, 6.000000000000000000, 50.00000000000000000 ) ) ;\n",
 "#401 = LINE ( 'NONE', #410, #323 ) ;\n",
 "#410 = CARTESIAN_POINT ( 'NONE',  ( -50.00000000000000000, 6.000000000000000000, -50.00000000000000000 ) ) ;\n",
 "#323 = VECTOR ( 'NONE', #81, 1000.000000000000000 ) ;\n",
 "#81 = DIRECTION ( 'NONE',  ( 1.387778780781445676E-16, 0.000000000000000000, 1.000000000000000000 ) ) ;\n",
 "#197 = ORIENTED_EDGE ( 'NONE', *, *, #133, .T. ) ;\n",
 "#133 = EDGE_CURVE ( 'NONE', #411, #82, #99, .T. ) ;\n",
 "#411 

In [36]:
edge_curve_outer_bond_data = get_specific_element(outer_boundary_data,"EDGE_CURVE")
edge_curve_outer_bond_data

array(["#271 = EDGE_CURVE ( 'NONE', #407, #411, #401, .T. ) ;",
       "#133 = EDGE_CURVE ( 'NONE', #411, #82, #99, .T. ) ;",
       "#376 = EDGE_CURVE ( 'NONE', #82, #150, #361, .T. ) ;",
       "#301 = EDGE_CURVE ( 'NONE', #150, #407, #288, .T. ) ;"],
      dtype='<U53')

In [37]:
def extract_outer_face_data(step_array,face):
    print(face)
    output_array= get_extracted_data(step_array,[face])
    outer_boundary =  get_specific_element(output_array,"FACE_OUTER_BOUND")
    outer_boundary_data=get_extracted_data(step_array,outer_boundary)
    edge_curve_outer_bond_data = get_specific_element(outer_boundary_data,"EDGE_CURVE")
    face_lst=[]
    for edge in edge_curve_outer_bond_data:
        edge_data = get_inside_hashes(edge)
        edge_lst = []
        for e in edge_data:
            e_line = get_line_by_hash(step_array,e)
            element_name = get_element_name(e_line)
            edge_lst.append(element_name)
            if element_name =="LINE":
                pass
            if element_name == "CIRCLE":
                #get center point and radius
                center_point = get_circle_center_point(e_line)
                edge_lst+=center_point
                radius = get_circle_radius(e_line)
                edge_lst.append(radius)
                
                
            if element_name =="VERTEX_POINT":
                #get cartisean in this element
                #store it in the lst 
                element_data = get_extracted_data(step_array,[e_line])
                # print(element_data)
                cart_line_array= get_specific_element(element_data,"CARTESIAN_POINT ")
                for cart in cart_line_array:
                    points = extract_numbers_from_string(cart)
                    edge_lst+=points 
                    
        face_lst.append(edge_lst)
        
    return face_lst


In [38]:
def extract_inner_face_data(step_array,face):
    print(face)
    output_array= get_extracted_data(step_array,[face])
    outer_boundary =  get_specific_element(output_array,"FACE_BOUND")
    outer_boundary_data=get_extracted_data(step_array,outer_boundary)
    edge_curve_outer_bond_data = get_specific_element(outer_boundary_data,"EDGE_CURVE")
    face_lst=[]
    for edge in edge_curve_outer_bond_data:
        edge_data = get_inside_hashes(edge)
        edge_lst = []
        for e in edge_data:
            e_line = get_line_by_hash(step_array,e)
            element_name = get_element_name(e_line)
            edge_lst.append(element_name)
            if element_name =="LINE":
                pass
            if element_name == "CIRCLE":
                #get center point and radius
                center_point = get_circle_center_point(e_line)
                edge_lst+=center_point
                radius = get_circle_radius(e_line)
                edge_lst.append(radius)
                    
            if element_name =="VERTEX_POINT":
                #get cartisean in this element
                #store it in the lst 
                element_data = get_extracted_data(step_array,[e_line])
                # print(element_data)
                cart_line_array= get_specific_element(element_data,"CARTESIAN_POINT")
                for cart in cart_line_array:
                    points = extract_numbers_from_string(cart)
                    edge_lst+=points 
                    
        face_lst.append(edge_lst)
        
    return face_lst


In [39]:
outer_contour_data = extract_outer_face_data(step_array,major_face)
outer_contour_data

#216 = ADVANCED_FACE ( 'NONE', ( #149, #151, #277 ), #59, .T. ) ;


[['VERTEX_POINT', -50, 6, -50, 'VERTEX_POINT', -50, 6, 50, 'LINE'],
 ['VERTEX_POINT', -50, 6, 50, 'VERTEX_POINT', 150, 6, 50, 'LINE'],
 ['VERTEX_POINT', 150, 6, 50, 'VERTEX_POINT', 150, 6, -50, 'LINE'],
 ['VERTEX_POINT', 150, 6, -50, 'VERTEX_POINT', -50, 6, -50, 'LINE']]

In [41]:
print(major_face)

#216 = ADVANCED_FACE ( 'NONE', ( #149, #151, #277 ), #59, .T. ) ;


In [43]:
inner_boundaries =  get_specific_element(output_array,"FACE_BOUND")
inner_boundaries

array(["#151 = FACE_BOUND ( 'NONE', #307, .T. ) ;",
       "#277 = FACE_BOUND ( 'NONE', #315, .T. ) ;"], dtype='<U41')

In [44]:
inner_contour1 = inner_boundaries[0]
inner_contour1

"#151 = FACE_BOUND ( 'NONE', #307, .T. ) ;"

In [46]:
inner_contour2 = inner_boundaries[1]
inner_contour2

"#277 = FACE_BOUND ( 'NONE', #315, .T. ) ;"

In [50]:
inner_contour1_data = extract_inner_face_data(step_array,inner_contour1)
for l in inner_contour1_data:
    print(l)

#151 = FACE_BOUND ( 'NONE', #307, .T. ) ;
['VERTEX_POINT', -25, 6, -38, 'VERTEX_POINT', -25, 6, -22, 'CIRCLE', -25, 6, -30, 7.5]
['VERTEX_POINT', -25, 6, -22, 'VERTEX_POINT', -25, 6, -38, 'CIRCLE', -25, 6, -30, 7.5]


In [51]:
inner_contour2_data = extract_inner_face_data(step_array,inner_contour2)
for l in inner_contour2_data:
    print(l)

#277 = FACE_BOUND ( 'NONE', #315, .T. ) ;
['VERTEX_POINT', 110, 6, 45, 'VERTEX_POINT', 125, 6, 45, 'LINE']
['VERTEX_POINT', 110, 6, 30, 'VERTEX_POINT', 110, 6, 45, 'LINE']
['VERTEX_POINT', 125, 6, 30, 'VERTEX_POINT', 110, 6, 30, 'LINE']
['VERTEX_POINT', 125, 6, 45, 'VERTEX_POINT', 125, 6, 30, 'LINE']


In [62]:
# function to return data for inner contour
def get_inner_contours_data(step_array,major_face):
    inner_boundaries =  get_specific_element(output_array,"FACE_BOUND")
    contours_data =[]
    for contour in inner_boundaries:
        contours_data.append(extract_inner_face_data(step_array,contour))

    return contours_data

In [63]:
c_data = get_inner_contours_data(step_array, major_face)

#151 = FACE_BOUND ( 'NONE', #307, .T. ) ;
#277 = FACE_BOUND ( 'NONE', #315, .T. ) ;


In [64]:
for contour in c_data:
    for entity in contour:
        print(entity)

['VERTEX_POINT', -25, 6, -38, 'VERTEX_POINT', -25, 6, -22, 'CIRCLE', -25, 6, -30, 7.5]
['VERTEX_POINT', -25, 6, -22, 'VERTEX_POINT', -25, 6, -38, 'CIRCLE', -25, 6, -30, 7.5]
['VERTEX_POINT', 110, 6, 45, 'VERTEX_POINT', 125, 6, 45, 'LINE']
['VERTEX_POINT', 110, 6, 30, 'VERTEX_POINT', 110, 6, 45, 'LINE']
['VERTEX_POINT', 125, 6, 30, 'VERTEX_POINT', 110, 6, 30, 'LINE']
['VERTEX_POINT', 125, 6, 45, 'VERTEX_POINT', 125, 6, 30, 'LINE']
